In [1]:
import kagglehub

# Download latest version
base_dir = kagglehub.dataset_download("sartajbhuvaji/brain-tumor-classification-mri")

print("Path to dataset files:", base_dir)

Using Colab cache for faster access to the 'brain-tumor-classification-mri' dataset.
Path to dataset files: /kaggle/input/brain-tumor-classification-mri


In [2]:
base_dir

'/kaggle/input/brain-tumor-classification-mri'

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3

In [5]:
train_path = base_dir + '/Training'
test_path = base_dir + '/Testing'

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [6]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [7]:
import numpy as np

X_train = []
y_train = []

for images, labels in train_ds:
    X_train.extend(images.numpy())
    y_train.extend(labels.numpy())



for images, labels in test_ds:
    X_train.extend(images.numpy())
    y_train.extend(labels.numpy())
X_train = np.array(X_train)
y_train = np.array(y_train)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.1,random_state=42)

In [9]:
X_train.shape

(2937, 224, 224, 3)

In [10]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train,y_train, random_state=101)

In [11]:
X_train = X_train/255.0
X_test = X_test/255.0

In [12]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [13]:
from keras.applications import EfficientNetB0

In [14]:
effnet = EfficientNetB0(input_shape=(224,224,3),weights='imagenet',include_top=False)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [15]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 4,012,672 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [17]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [19]:
print(X_train.shape, y_train.shape)  # e.g., (2870, 224, 224, 3) (2870, 4)
print(X_test.shape, y_test.shape)

(2937, 224, 224, 3) (2937, 4)
(327, 224, 224, 3) (327, 4)


In [20]:
model.fit(X_train,y_train,epochs = 20,batch_size = 32,validation_split=0.1,callbacks=[tensorboard,checkpoint,reduce_lr])

Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 741ms/step - accuracy: 0.7322 - loss: 0.6519
Epoch 1: val_accuracy improved from -inf to 0.27211, saving model to effnet.h5


83/83 ━━━━━━━━━━━━━━━━━━━━ 175s 1000ms/step - accuracy: 0.7333 - loss: 0.6497 - val_accuracy: 0.2721 - val_loss: 1.5720 - learning_rate: 0.0010
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9264 - loss: 0.2009
Epoch 2: val_accuracy did not improve from 0.27211
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.9266 - loss: 0.2006 - val_accuracy: 0.1259 - val_loss: 2.0327 - learning_rate: 0.0010
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9535 - loss: 0.1296
Epoch 3: val_accuracy improved from 0.27211 to 0.27891, saving model to effnet.h5


83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - accuracy: 0.9535 - loss: 0.1296 - val_accuracy: 0.2789 - val_loss: 1.4029 - learning_rate: 0.0010
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9742 - loss: 0.0882
Epoch 4: val_accuracy did not improve from 0.27891
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 131ms/step - accuracy: 0.9742 - loss: 0.0883 - val_accuracy: 0.1122 - val_loss: 22.7681 - learning_rate: 0.0010
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9804 - loss: 0.0550
Epoch 5: val_accuracy did not improve from 0.27891

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 132ms/step - accuracy: 0.9804 - loss: 0.0551 - val_accuracy: 0.1259 - val_loss: 4.1677 - learning_rate: 0.0010
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9899 - loss: 0.0255
Epoch 6: val_accuracy did not improve from 0.27891
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 132ms/step - accuracy: 0.9899 - loss: 0.0254 - val_acc

83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - accuracy: 0.9977 - loss: 0.0095 - val_accuracy: 0.7891 - val_loss: 0.8043 - learning_rate: 9.0000e-05
Epoch 9/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9985 - loss: 0.0056
Epoch 9: val_accuracy did not improve from 0.78912
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.9985 - loss: 0.0056 - val_accuracy: 0.6293 - val_loss: 1.3329 - learning_rate: 9.0000e-05
Epoch 10/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9989 - loss: 0.0047
Epoch 10: val_accuracy did not improve from 0.78912

Epoch 10: ReduceLROnPlateau reducing learning rate to 2.700000040931627e-05.
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.9989 - loss: 0.0047 - val_accuracy: 0.7109 - val_loss: 0.7756 - learning_rate: 9.0000e-05
Epoch 11/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9988 - loss: 0.0059
Epoch 11: val_accuracy improved from 0.78912 to 0.92177, saving model to effnet.h5


83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - accuracy: 0.9988 - loss: 0.0059 - val_accuracy: 0.9218 - val_loss: 0.2757 - learning_rate: 2.7000e-05
Epoch 12/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9984 - loss: 0.0056
Epoch 12: val_accuracy improved from 0.92177 to 0.96599, saving model to effnet.h5


83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - accuracy: 0.9984 - loss: 0.0056 - val_accuracy: 0.9660 - val_loss: 0.1749 - learning_rate: 2.7000e-05
Epoch 13/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9978 - loss: 0.0071
Epoch 13: val_accuracy improved from 0.96599 to 0.96939, saving model to effnet.h5


83/83 ━━━━━━━━━━━━━━━━━━━━ 12s 139ms/step - accuracy: 0.9978 - loss: 0.0071 - val_accuracy: 0.9694 - val_loss: 0.1894 - learning_rate: 2.7000e-05
Epoch 14/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9995 - loss: 0.0032
Epoch 14: val_accuracy did not improve from 0.96939
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.9995 - loss: 0.0032 - val_accuracy: 0.9490 - val_loss: 0.2542 - learning_rate: 2.7000e-05
Epoch 15/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9997 - loss: 0.0029
Epoch 15: val_accuracy did not improve from 0.96939

Epoch 15: ReduceLROnPlateau reducing learning rate to 8.100000013655517e-06.
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9422 - val_loss: 0.3023 - learning_rate: 2.7000e-05
Epoch 16/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9998 - loss: 0.0037
Epoch 16: val_accuracy did not improve from 0.96939
83/83 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.9998 - loss

In [21]:
acc,loss = model.evaluate(X_test,y_test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9792 - loss: 0.1265


In [25]:
import keras
keras.saving.save_model(model, 'my_model.keras')